In [2]:
import pandas as pd
import re
import numpy as np
import nltk
import string
import fasttext
import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
plt.xticks(rotation=70)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)
%matplotlib inline

In [5]:
data = pd.read_csv('E:\MATERI PERKULIAHAN TINGKAT 4\Data Mining\proyek\dataset/mq_news.csv')

In [7]:
data[:10]

,Unnamed: 0,item_id,url,title,content_html,published_at,source
0,0,5af9bdef421aa98c745d28cf,https://www.liputan6.com/news/read/2691369/tips-agar-resolusi-tahun-baru-bisa-menjadi-aksi-nyata,Tips Agar Resolusi Tahun Baru Bisa Menjadi Aksi Nyata,"<div class=""article-content-body__item-page "" data-page=""1"" data-title=""""><div class=""article-co...",2017-01-01 00:00:00,liputan6.com
1,1,5af9bdeb421aa98c745d28ce,https://www.liputan6.com/news/read/2692804/tragedi-mendebarkan-di-penghujung-2016,Tragedi Mendebarkan di Penghujung 2016,"<div class=""article-content-body__item-page "" data-page=""1"" data-title=""""><div class=""article-co...",2017-01-01 00:01:00,liputan6.com
2,2,5af85c6b421aa922e201b8e2,http://megapolitan.kompas.com/read/2017/01/01/00030941/pantau.tahun.baru.kapolda.metro.jaya.bonc...,"Pantau Tahun Baru, Kapolda Metro Jaya Bonceng Anak Pakai Motor Trail","<p><strong>JAKARTA, KOMPAS.com</strong> - Kapolda Metro Jaya Irjen Mochamad Iriawan memantau lan...",2017-01-01 00:03:09,kompas.com
3,3,5af9bde7421aa98c745d28cd,https://www.liputan6.com/news/read/2693107/terompet-dan-kembang-api-bersahutan-sambut-tahun-baru...,Terompet dan Kembang Api Bersahutan Sambut Tahun Baru di Monas,"<div class=""article-content-body__item-page "" data-page=""1"" data-title=""""><div class=""article-co...",2017-01-01 00:04:21,liputan6.com
4,4,5af9bde4421aa98c745d28cc,https://www.liputan6.com/news/read/2693110/awas-copet-berkeliaran-di-ancol-malam-tahun-baru-ini,Awas! Copet Berkeliaran di Ancol Malam Tahun Baru Ini,"<div class=""article-content-body__item-page "" data-page=""1"" data-title=""""><div class=""article-co...",2017-01-01 00:05:08,liputan6.com
5,5,5af85294421aa903c108e669,http://www.tribunnews.com/superskor/2017/01/01/dua-gol-kejutan-manchester-united-di-menit-akhir-...,Dua Gol Kejutan Manchester United di Menit Akhir Kandaskan Middlesbrough,<p><strong>TRIBUNNEWS.COM - </strong>Manchester United mendulang poin sempurna atas Middlesbroug...,2017-01-01 00:05:54,tribunnews.com
6,6,5af85290421aa903c108e668,http://www.tribunnews.com/superskor/2017/01/01/diego-costa-cetak-gol-pamungkas-ke-jala-stoke-cit...,"Diego Costa Cetak Gol Pamungkas ke Jala Stoke City, Chelsea Perkasa dengan 49 Poin","<p><strong>TRIBUNNEWS.COM - </strong>Diego Costa melengkapi gol kemenangan <a href=""http://www.t...",2017-01-01 00:16:52,tribunnews.com
7,7,5af85c65421aa922e201b8e1,http://regional.kompas.com/read/2017/01/01/00165411/rayakan.malam.pergantian.tahun.warga.ambon.p...,"Rayakan Malam Pergantian Tahun, Warga Ambon Padati Jembatan Merah Putih","<p><strong>AMBON,KOMPAS.com - </strong>Warga dari berbagai penjuru di Kota Ambon tumpah ruah di ...",2017-01-01 00:16:54,kompas.com
8,8,5ae09a2b421aa96432723007,https://news.detik.com/berita/d-3385312/plt-gubernur-dki-tembakkan-suar-tahun-baru-2017,Plt Gubernur DKI Tembakkan Suar Tahun Baru 2017,<b>Jakarta</b> -Plt Gubernur DKI Jakarta Sumarsono menghadiri perayaan tahun baru 2017 di panggu...,2017-01-01 00:20:52,detik.com
9,9,5af85c60421aa922e201b8e0,http://regional.kompas.com/read/2017/01/01/00243771/klakson.telolet.ramaikan.malam.tahun.baru.di...,"Klakson ""Telolet"" Ramaikan Malam Tahun Baru di Surabaya","<p><strong>SURABAYA, KOMPAS.com</strong> - Detik-detik pergantian tahun tidak hanya diramaikan o...",2017-01-01 00:24:37,kompas.com


In [47]:
data.shape

(455678, 7)

In [8]:
#mengambil atribut pada kolom 2-4 (1:4) (menggunakan rentang/interval)
#att_train = data_train.iloc[:, 1:4]
#mengambil atribut "title" dan "text" (menggunakan nama atribut)
data2 =data.loc[:, ['item_id','title', 'source']]

In [9]:
data2[:11]

,item_id,title,source
0,5af9bdef421aa98c745d28cf,Tips Agar Resolusi Tahun Baru Bisa Menjadi Aksi Nyata,liputan6.com
1,5af9bdeb421aa98c745d28ce,Tragedi Mendebarkan di Penghujung 2016,liputan6.com
2,5af85c6b421aa922e201b8e2,"Pantau Tahun Baru, Kapolda Metro Jaya Bonceng Anak Pakai Motor Trail",kompas.com
3,5af9bde7421aa98c745d28cd,Terompet dan Kembang Api Bersahutan Sambut Tahun Baru di Monas,liputan6.com
4,5af9bde4421aa98c745d28cc,Awas! Copet Berkeliaran di Ancol Malam Tahun Baru Ini,liputan6.com
5,5af85294421aa903c108e669,Dua Gol Kejutan Manchester United di Menit Akhir Kandaskan Middlesbrough,tribunnews.com
6,5af85290421aa903c108e668,"Diego Costa Cetak Gol Pamungkas ke Jala Stoke City, Chelsea Perkasa dengan 49 Poin",tribunnews.com
7,5af85c65421aa922e201b8e1,"Rayakan Malam Pergantian Tahun, Warga Ambon Padati Jembatan Merah Putih",kompas.com
8,5ae09a2b421aa96432723007,Plt Gubernur DKI Tembakkan Suar Tahun Baru 2017,detik.com
9,5af85c60421aa922e201b8e0,"Klakson ""Telolet"" Ramaikan Malam Tahun Baru di Surabaya",kompas.com


In [10]:
#mengecek nilai null pada kolom id, title dan source
for col in data2.columns:
    print(col, data2[col].isnull().sum())

item_id 0
title 0
source 0


In [11]:
data3 = data2.dropna()

In [12]:
#mengecek nilai null pada kolom id dan title dan source
for col in data3.columns:
    print(col, data3[col].isnull().sum())

item_id 0
title 0
source 0


In [13]:
data3.shape

(455678, 3)

In [14]:
#menghapus angka
data3['title'] = data3['title'].str.replace('\d+', '')

In [15]:
data3[:11]

,item_id,title,source
0,5af9bdef421aa98c745d28cf,Tips Agar Resolusi Tahun Baru Bisa Menjadi Aksi Nyata,liputan6.com
1,5af9bdeb421aa98c745d28ce,Tragedi Mendebarkan di Penghujung,liputan6.com
2,5af85c6b421aa922e201b8e2,"Pantau Tahun Baru, Kapolda Metro Jaya Bonceng Anak Pakai Motor Trail",kompas.com
3,5af9bde7421aa98c745d28cd,Terompet dan Kembang Api Bersahutan Sambut Tahun Baru di Monas,liputan6.com
4,5af9bde4421aa98c745d28cc,Awas! Copet Berkeliaran di Ancol Malam Tahun Baru Ini,liputan6.com
5,5af85294421aa903c108e669,Dua Gol Kejutan Manchester United di Menit Akhir Kandaskan Middlesbrough,tribunnews.com
6,5af85290421aa903c108e668,"Diego Costa Cetak Gol Pamungkas ke Jala Stoke City, Chelsea Perkasa dengan Poin",tribunnews.com
7,5af85c65421aa922e201b8e1,"Rayakan Malam Pergantian Tahun, Warga Ambon Padati Jembatan Merah Putih",kompas.com
8,5ae09a2b421aa96432723007,Plt Gubernur DKI Tembakkan Suar Tahun Baru,detik.com
9,5af85c60421aa922e201b8e0,"Klakson ""Telolet"" Ramaikan Malam Tahun Baru di Surabaya",kompas.com


In [16]:
#melakukan contraction (mengembalikan singkatan ke bentuk awalnya) contoh: you're => you are
data3['no_contract'] = data3['title'].apply(lambda x: [contractions.fix(word) for word in x.split()])

In [17]:
data3[:11]

,item_id,title,source,no_contract
0,5af9bdef421aa98c745d28cf,Tips Agar Resolusi Tahun Baru Bisa Menjadi Aksi Nyata,liputan6.com,"[Tips, Agar, Resolusi, Tahun, Baru, Bisa, Menjadi, Aksi, Nyata]"
1,5af9bdeb421aa98c745d28ce,Tragedi Mendebarkan di Penghujung,liputan6.com,"[Tragedi, Mendebarkan, di, Penghujung]"
2,5af85c6b421aa922e201b8e2,"Pantau Tahun Baru, Kapolda Metro Jaya Bonceng Anak Pakai Motor Trail",kompas.com,"[Pantau, Tahun, Baru,, Kapolda, Metro, Jaya, Bonceng, Anak, Pakai, Motor, Trail]"
3,5af9bde7421aa98c745d28cd,Terompet dan Kembang Api Bersahutan Sambut Tahun Baru di Monas,liputan6.com,"[Terompet, dan, Kembang, Api, Bersahutan, Sambut, Tahun, Baru, di, Monas]"
4,5af9bde4421aa98c745d28cc,Awas! Copet Berkeliaran di Ancol Malam Tahun Baru Ini,liputan6.com,"[Awas!, Copet, Berkeliaran, di, Ancol, Malam, Tahun, Baru, Ini]"
5,5af85294421aa903c108e669,Dua Gol Kejutan Manchester United di Menit Akhir Kandaskan Middlesbrough,tribunnews.com,"[Dua, Gol, Kejutan, Manchester, United, di, Menit, Akhir, Kandaskan, Middlesbrough]"
6,5af85290421aa903c108e668,"Diego Costa Cetak Gol Pamungkas ke Jala Stoke City, Chelsea Perkasa dengan Poin",tribunnews.com,"[Diego, Costa, Cetak, Gol, Pamungkas, ke, Jala, Stoke, City,, Chelsea, Perkasa, dengan, Poin]"
7,5af85c65421aa922e201b8e1,"Rayakan Malam Pergantian Tahun, Warga Ambon Padati Jembatan Merah Putih",kompas.com,"[Rayakan, Malam, Pergantian, Tahun,, Warga, Ambon, Padati, Jembatan, Merah, Putih]"
8,5ae09a2b421aa96432723007,Plt Gubernur DKI Tembakkan Suar Tahun Baru,detik.com,"[Plt, Gubernur, DKI, Tembakkan, Suar, Tahun, Baru]"
9,5af85c60421aa922e201b8e0,"Klakson ""Telolet"" Ramaikan Malam Tahun Baru di Surabaya",kompas.com,"[Klakson, ""Telolet"", Ramaikan, Malam, Tahun, Baru, di, Surabaya]"


In [18]:
#mengembalikan kembali teks yang dispit dan ditampung oleh attribut title_str
data3['title_str'] = [' '.join(map(str, l)) for l in data3['no_contract']]
data3.head()

,item_id,title,source,no_contract,title_str
0,5af9bdef421aa98c745d28cf,Tips Agar Resolusi Tahun Baru Bisa Menjadi Aksi Nyata,liputan6.com,"[Tips, Agar, Resolusi, Tahun, Baru, Bisa, Menjadi, Aksi, Nyata]",Tips Agar Resolusi Tahun Baru Bisa Menjadi Aksi Nyata
1,5af9bdeb421aa98c745d28ce,Tragedi Mendebarkan di Penghujung,liputan6.com,"[Tragedi, Mendebarkan, di, Penghujung]",Tragedi Mendebarkan di Penghujung
2,5af85c6b421aa922e201b8e2,"Pantau Tahun Baru, Kapolda Metro Jaya Bonceng Anak Pakai Motor Trail",kompas.com,"[Pantau, Tahun, Baru,, Kapolda, Metro, Jaya, Bonceng, Anak, Pakai, Motor, Trail]","Pantau Tahun Baru, Kapolda Metro Jaya Bonceng Anak Pakai Motor Trail"
3,5af9bde7421aa98c745d28cd,Terompet dan Kembang Api Bersahutan Sambut Tahun Baru di Monas,liputan6.com,"[Terompet, dan, Kembang, Api, Bersahutan, Sambut, Tahun, Baru, di, Monas]",Terompet dan Kembang Api Bersahutan Sambut Tahun Baru di Monas
4,5af9bde4421aa98c745d28cc,Awas! Copet Berkeliaran di Ancol Malam Tahun Baru Ini,liputan6.com,"[Awas!, Copet, Berkeliaran, di, Ancol, Malam, Tahun, Baru, Ini]",Awas! Copet Berkeliaran di Ancol Malam Tahun Baru Ini


## Tokenization

In [19]:
#melakukan tokenisasi pada atribut "title_str" dan ditampung atribut baru "tokenized"
data3['tokenized'] = data3['title_str'].apply(word_tokenize)
data3.head()

,item_id,title,source,no_contract,title_str,tokenized
0,5af9bdef421aa98c745d28cf,Tips Agar Resolusi Tahun Baru Bisa Menjadi Aksi Nyata,liputan6.com,"[Tips, Agar, Resolusi, Tahun, Baru, Bisa, Menjadi, Aksi, Nyata]",Tips Agar Resolusi Tahun Baru Bisa Menjadi Aksi Nyata,"[Tips, Agar, Resolusi, Tahun, Baru, Bisa, Menjadi, Aksi, Nyata]"
1,5af9bdeb421aa98c745d28ce,Tragedi Mendebarkan di Penghujung,liputan6.com,"[Tragedi, Mendebarkan, di, Penghujung]",Tragedi Mendebarkan di Penghujung,"[Tragedi, Mendebarkan, di, Penghujung]"
2,5af85c6b421aa922e201b8e2,"Pantau Tahun Baru, Kapolda Metro Jaya Bonceng Anak Pakai Motor Trail",kompas.com,"[Pantau, Tahun, Baru,, Kapolda, Metro, Jaya, Bonceng, Anak, Pakai, Motor, Trail]","Pantau Tahun Baru, Kapolda Metro Jaya Bonceng Anak Pakai Motor Trail","[Pantau, Tahun, Baru, ,, Kapolda, Metro, Jaya, Bonceng, Anak, Pakai, Motor, Trail]"
3,5af9bde7421aa98c745d28cd,Terompet dan Kembang Api Bersahutan Sambut Tahun Baru di Monas,liputan6.com,"[Terompet, dan, Kembang, Api, Bersahutan, Sambut, Tahun, Baru, di, Monas]",Terompet dan Kembang Api Bersahutan Sambut Tahun Baru di Monas,"[Terompet, dan, Kembang, Api, Bersahutan, Sambut, Tahun, Baru, di, Monas]"
4,5af9bde4421aa98c745d28cc,Awas! Copet Berkeliaran di Ancol Malam Tahun Baru Ini,liputan6.com,"[Awas!, Copet, Berkeliaran, di, Ancol, Malam, Tahun, Baru, Ini]",Awas! Copet Berkeliaran di Ancol Malam Tahun Baru Ini,"[Awas, !, Copet, Berkeliaran, di, Ancol, Malam, Tahun, Baru, Ini]"


## Converting all characters to lower case

In [78]:
#mengubah semua huruf besar menjadi huruf kecil dan ditampung pada atribut baru "lower"
data3['lower'] = data3['tokenized'].apply(lambda x: [word.lower() for word in x])
data3.head()

,item_id,title,source,no_contract,title_str,tokenized,lower,no_punc,stopwords_removed
0,5af9bdef421aa98c745d28cf,Tips Agar Resolusi Tahun Baru Bisa Menjadi Aksi Nyata,liputan6.com,"[Tips, Agar, Resolusi, Tahun, Baru, Bisa, Menjadi, Aksi, Nyata]",Tips Agar Resolusi Tahun Baru Bisa Menjadi Aksi Nyata,"[Tips, Agar, Resolusi, Tahun, Baru, Bisa, Menjadi, Aksi, Nyata]","[tips, agar, resolusi, tahun, baru, bisa, menjadi, aksi, nyata]","[tips, agar, resolusi, tahun, baru, bisa, menjadi, aksi, nyata]","[tips, resolusi, aksi, nyata]"
1,5af9bdeb421aa98c745d28ce,Tragedi Mendebarkan di Penghujung,liputan6.com,"[Tragedi, Mendebarkan, di, Penghujung]",Tragedi Mendebarkan di Penghujung,"[Tragedi, Mendebarkan, di, Penghujung]","[tragedi, mendebarkan, di, penghujung]","[tragedi, mendebarkan, di, penghujung]","[tragedi, mendebarkan, penghujung]"
2,5af85c6b421aa922e201b8e2,"Pantau Tahun Baru, Kapolda Metro Jaya Bonceng Anak Pakai Motor Trail",kompas.com,"[Pantau, Tahun, Baru,, Kapolda, Metro, Jaya, Bonceng, Anak, Pakai, Motor, Trail]","Pantau Tahun Baru, Kapolda Metro Jaya Bonceng Anak Pakai Motor Trail","[Pantau, Tahun, Baru, ,, Kapolda, Metro, Jaya, Bonceng, Anak, Pakai, Motor, Trail]","[pantau, tahun, baru, ,, kapolda, metro, jaya, bonceng, anak, pakai, motor, trail]","[pantau, tahun, baru, kapolda, metro, jaya, bonceng, anak, pakai, motor, trail]","[pantau, kapolda, metro, jaya, bonceng, anak, pakai, motor, trail]"
3,5af9bde7421aa98c745d28cd,Terompet dan Kembang Api Bersahutan Sambut Tahun Baru di Monas,liputan6.com,"[Terompet, dan, Kembang, Api, Bersahutan, Sambut, Tahun, Baru, di, Monas]",Terompet dan Kembang Api Bersahutan Sambut Tahun Baru di Monas,"[Terompet, dan, Kembang, Api, Bersahutan, Sambut, Tahun, Baru, di, Monas]","[terompet, dan, kembang, api, bersahutan, sambut, tahun, baru, di, monas]","[terompet, dan, kembang, api, bersahutan, sambut, tahun, baru, di, monas]","[terompet, kembang, api, bersahutan, sambut, monas]"
4,5af9bde4421aa98c745d28cc,Awas! Copet Berkeliaran di Ancol Malam Tahun Baru Ini,liputan6.com,"[Awas!, Copet, Berkeliaran, di, Ancol, Malam, Tahun, Baru, Ini]",Awas! Copet Berkeliaran di Ancol Malam Tahun Baru Ini,"[Awas, !, Copet, Berkeliaran, di, Ancol, Malam, Tahun, Baru, Ini]","[awas, !, copet, berkeliaran, di, ancol, malam, tahun, baru, ini]","[awas, copet, berkeliaran, di, ancol, malam, tahun, baru, ini]","[awas, copet, berkeliaran, ancol, malam]"


## Removing Punctuations

In [79]:
#menghilangkan tanda baca dan ditampung oleh atribut baru yaitu "no_punc"
punc = string.punctuation
data3['no_punc'] = data3['lower'].apply(lambda x: [word for word in x if word not in punc])
data3.head()

,item_id,title,source,no_contract,title_str,tokenized,lower,no_punc,stopwords_removed
0,5af9bdef421aa98c745d28cf,Tips Agar Resolusi Tahun Baru Bisa Menjadi Aksi Nyata,liputan6.com,"[Tips, Agar, Resolusi, Tahun, Baru, Bisa, Menjadi, Aksi, Nyata]",Tips Agar Resolusi Tahun Baru Bisa Menjadi Aksi Nyata,"[Tips, Agar, Resolusi, Tahun, Baru, Bisa, Menjadi, Aksi, Nyata]","[tips, agar, resolusi, tahun, baru, bisa, menjadi, aksi, nyata]","[tips, agar, resolusi, tahun, baru, bisa, menjadi, aksi, nyata]","[tips, resolusi, aksi, nyata]"
1,5af9bdeb421aa98c745d28ce,Tragedi Mendebarkan di Penghujung,liputan6.com,"[Tragedi, Mendebarkan, di, Penghujung]",Tragedi Mendebarkan di Penghujung,"[Tragedi, Mendebarkan, di, Penghujung]","[tragedi, mendebarkan, di, penghujung]","[tragedi, mendebarkan, di, penghujung]","[tragedi, mendebarkan, penghujung]"
2,5af85c6b421aa922e201b8e2,"Pantau Tahun Baru, Kapolda Metro Jaya Bonceng Anak Pakai Motor Trail",kompas.com,"[Pantau, Tahun, Baru,, Kapolda, Metro, Jaya, Bonceng, Anak, Pakai, Motor, Trail]","Pantau Tahun Baru, Kapolda Metro Jaya Bonceng Anak Pakai Motor Trail","[Pantau, Tahun, Baru, ,, Kapolda, Metro, Jaya, Bonceng, Anak, Pakai, Motor, Trail]","[pantau, tahun, baru, ,, kapolda, metro, jaya, bonceng, anak, pakai, motor, trail]","[pantau, tahun, baru, kapolda, metro, jaya, bonceng, anak, pakai, motor, trail]","[pantau, kapolda, metro, jaya, bonceng, anak, pakai, motor, trail]"
3,5af9bde7421aa98c745d28cd,Terompet dan Kembang Api Bersahutan Sambut Tahun Baru di Monas,liputan6.com,"[Terompet, dan, Kembang, Api, Bersahutan, Sambut, Tahun, Baru, di, Monas]",Terompet dan Kembang Api Bersahutan Sambut Tahun Baru di Monas,"[Terompet, dan, Kembang, Api, Bersahutan, Sambut, Tahun, Baru, di, Monas]","[terompet, dan, kembang, api, bersahutan, sambut, tahun, baru, di, monas]","[terompet, dan, kembang, api, bersahutan, sambut, tahun, baru, di, monas]","[terompet, kembang, api, bersahutan, sambut, monas]"
4,5af9bde4421aa98c745d28cc,Awas! Copet Berkeliaran di Ancol Malam Tahun Baru Ini,liputan6.com,"[Awas!, Copet, Berkeliaran, di, Ancol, Malam, Tahun, Baru, Ini]",Awas! Copet Berkeliaran di Ancol Malam Tahun Baru Ini,"[Awas, !, Copet, Berkeliaran, di, Ancol, Malam, Tahun, Baru, Ini]","[awas, !, copet, berkeliaran, di, ancol, malam, tahun, baru, ini]","[awas, copet, berkeliaran, di, ancol, malam, tahun, baru, ini]","[awas, copet, berkeliaran, ancol, malam]"


## Removing Stopwords

In [80]:
#menghilangkan kata kurang penting
stop_words = set(stopwords.words('indonesian'))
data3['stopwords_removed'] = data3['no_punc'].apply(lambda x: [word for word in x if word not in stop_words])
data3.head()

,item_id,title,source,no_contract,title_str,tokenized,lower,no_punc,stopwords_removed
0,5af9bdef421aa98c745d28cf,Tips Agar Resolusi Tahun Baru Bisa Menjadi Aksi Nyata,liputan6.com,"[Tips, Agar, Resolusi, Tahun, Baru, Bisa, Menjadi, Aksi, Nyata]",Tips Agar Resolusi Tahun Baru Bisa Menjadi Aksi Nyata,"[Tips, Agar, Resolusi, Tahun, Baru, Bisa, Menjadi, Aksi, Nyata]","[tips, agar, resolusi, tahun, baru, bisa, menjadi, aksi, nyata]","[tips, agar, resolusi, tahun, baru, bisa, menjadi, aksi, nyata]","[tips, resolusi, aksi, nyata]"
1,5af9bdeb421aa98c745d28ce,Tragedi Mendebarkan di Penghujung,liputan6.com,"[Tragedi, Mendebarkan, di, Penghujung]",Tragedi Mendebarkan di Penghujung,"[Tragedi, Mendebarkan, di, Penghujung]","[tragedi, mendebarkan, di, penghujung]","[tragedi, mendebarkan, di, penghujung]","[tragedi, mendebarkan, penghujung]"
2,5af85c6b421aa922e201b8e2,"Pantau Tahun Baru, Kapolda Metro Jaya Bonceng Anak Pakai Motor Trail",kompas.com,"[Pantau, Tahun, Baru,, Kapolda, Metro, Jaya, Bonceng, Anak, Pakai, Motor, Trail]","Pantau Tahun Baru, Kapolda Metro Jaya Bonceng Anak Pakai Motor Trail","[Pantau, Tahun, Baru, ,, Kapolda, Metro, Jaya, Bonceng, Anak, Pakai, Motor, Trail]","[pantau, tahun, baru, ,, kapolda, metro, jaya, bonceng, anak, pakai, motor, trail]","[pantau, tahun, baru, kapolda, metro, jaya, bonceng, anak, pakai, motor, trail]","[pantau, kapolda, metro, jaya, bonceng, anak, pakai, motor, trail]"
3,5af9bde7421aa98c745d28cd,Terompet dan Kembang Api Bersahutan Sambut Tahun Baru di Monas,liputan6.com,"[Terompet, dan, Kembang, Api, Bersahutan, Sambut, Tahun, Baru, di, Monas]",Terompet dan Kembang Api Bersahutan Sambut Tahun Baru di Monas,"[Terompet, dan, Kembang, Api, Bersahutan, Sambut, Tahun, Baru, di, Monas]","[terompet, dan, kembang, api, bersahutan, sambut, tahun, baru, di, monas]","[terompet, dan, kembang, api, bersahutan, sambut, tahun, baru, di, monas]","[terompet, kembang, api, bersahutan, sambut, monas]"
4,5af9bde4421aa98c745d28cc,Awas! Copet Berkeliaran di Ancol Malam Tahun Baru Ini,liputan6.com,"[Awas!, Copet, Berkeliaran, di, Ancol, Malam, Tahun, Baru, Ini]",Awas! Copet Berkeliaran di Ancol Malam Tahun Baru Ini,"[Awas, !, Copet, Berkeliaran, di, Ancol, Malam, Tahun, Baru, Ini]","[awas, !, copet, berkeliaran, di, ancol, malam, tahun, baru, ini]","[awas, copet, berkeliaran, di, ancol, malam, tahun, baru, ini]","[awas, copet, berkeliaran, ancol, malam]"


## Stemming vs Lemmatization

In [83]:
#menerapkan metode part of speech tags dan ditampung atribut baru "pos_tags"
data3['pos_tags'] = data3['stopwords_removed'].apply(nltk.tag.pos_tag)
data3.head()

,item_id,title,source,no_contract,title_str,tokenized,lower,no_punc,stopwords_removed,pos_tags
0,5af9bdef421aa98c745d28cf,Tips Agar Resolusi Tahun Baru Bisa Menjadi Aksi Nyata,liputan6.com,"[Tips, Agar, Resolusi, Tahun, Baru, Bisa, Menjadi, Aksi, Nyata]",Tips Agar Resolusi Tahun Baru Bisa Menjadi Aksi Nyata,"[Tips, Agar, Resolusi, Tahun, Baru, Bisa, Menjadi, Aksi, Nyata]","[tips, agar, resolusi, tahun, baru, bisa, menjadi, aksi, nyata]","[tips, agar, resolusi, tahun, baru, bisa, menjadi, aksi, nyata]","[tips, resolusi, aksi, nyata]","[(tips, NNS), (resolusi, VBP), (aksi, NN), (nyata, NNS)]"
1,5af9bdeb421aa98c745d28ce,Tragedi Mendebarkan di Penghujung,liputan6.com,"[Tragedi, Mendebarkan, di, Penghujung]",Tragedi Mendebarkan di Penghujung,"[Tragedi, Mendebarkan, di, Penghujung]","[tragedi, mendebarkan, di, penghujung]","[tragedi, mendebarkan, di, penghujung]","[tragedi, mendebarkan, penghujung]","[(tragedi, NN), (mendebarkan, NN), (penghujung, NN)]"
2,5af85c6b421aa922e201b8e2,"Pantau Tahun Baru, Kapolda Metro Jaya Bonceng Anak Pakai Motor Trail",kompas.com,"[Pantau, Tahun, Baru,, Kapolda, Metro, Jaya, Bonceng, Anak, Pakai, Motor, Trail]","Pantau Tahun Baru, Kapolda Metro Jaya Bonceng Anak Pakai Motor Trail","[Pantau, Tahun, Baru, ,, Kapolda, Metro, Jaya, Bonceng, Anak, Pakai, Motor, Trail]","[pantau, tahun, baru, ,, kapolda, metro, jaya, bonceng, anak, pakai, motor, trail]","[pantau, tahun, baru, kapolda, metro, jaya, bonceng, anak, pakai, motor, trail]","[pantau, kapolda, metro, jaya, bonceng, anak, pakai, motor, trail]","[(pantau, NN), (kapolda, FW), (metro, NN), (jaya, NN), (bonceng, NN), (anak, NN), (pakai, NN), (..."
3,5af9bde7421aa98c745d28cd,Terompet dan Kembang Api Bersahutan Sambut Tahun Baru di Monas,liputan6.com,"[Terompet, dan, Kembang, Api, Bersahutan, Sambut, Tahun, Baru, di, Monas]",Terompet dan Kembang Api Bersahutan Sambut Tahun Baru di Monas,"[Terompet, dan, Kembang, Api, Bersahutan, Sambut, Tahun, Baru, di, Monas]","[terompet, dan, kembang, api, bersahutan, sambut, tahun, baru, di, monas]","[terompet, dan, kembang, api, bersahutan, sambut, tahun, baru, di, monas]","[terompet, kembang, api, bersahutan, sambut, monas]","[(terompet, NN), (kembang, VBD), (api, JJ), (bersahutan, NN), (sambut, NN), (monas, NNS)]"
4,5af9bde4421aa98c745d28cc,Awas! Copet Berkeliaran di Ancol Malam Tahun Baru Ini,liputan6.com,"[Awas!, Copet, Berkeliaran, di, Ancol, Malam, Tahun, Baru, Ini]",Awas! Copet Berkeliaran di Ancol Malam Tahun Baru Ini,"[Awas, !, Copet, Berkeliaran, di, Ancol, Malam, Tahun, Baru, Ini]","[awas, !, copet, berkeliaran, di, ancol, malam, tahun, baru, ini]","[awas, copet, berkeliaran, di, ancol, malam, tahun, baru, ini]","[awas, copet, berkeliaran, ancol, malam]","[(awas, NN), (copet, NN), (berkeliaran, NN), (ancol, NN), (malam, NN)]"


In [84]:
#mengganti part of speech tags 
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
data3['wordnet_pos'] = data3['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])

In [85]:
data3.head()

,item_id,title,source,no_contract,title_str,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos
0,5af9bdef421aa98c745d28cf,Tips Agar Resolusi Tahun Baru Bisa Menjadi Aksi Nyata,liputan6.com,"[Tips, Agar, Resolusi, Tahun, Baru, Bisa, Menjadi, Aksi, Nyata]",Tips Agar Resolusi Tahun Baru Bisa Menjadi Aksi Nyata,"[Tips, Agar, Resolusi, Tahun, Baru, Bisa, Menjadi, Aksi, Nyata]","[tips, agar, resolusi, tahun, baru, bisa, menjadi, aksi, nyata]","[tips, agar, resolusi, tahun, baru, bisa, menjadi, aksi, nyata]","[tips, resolusi, aksi, nyata]","[(tips, NNS), (resolusi, VBP), (aksi, NN), (nyata, NNS)]","[(tips, n), (resolusi, v), (aksi, n), (nyata, n)]"
1,5af9bdeb421aa98c745d28ce,Tragedi Mendebarkan di Penghujung,liputan6.com,"[Tragedi, Mendebarkan, di, Penghujung]",Tragedi Mendebarkan di Penghujung,"[Tragedi, Mendebarkan, di, Penghujung]","[tragedi, mendebarkan, di, penghujung]","[tragedi, mendebarkan, di, penghujung]","[tragedi, mendebarkan, penghujung]","[(tragedi, NN), (mendebarkan, NN), (penghujung, NN)]","[(tragedi, n), (mendebarkan, n), (penghujung, n)]"
2,5af85c6b421aa922e201b8e2,"Pantau Tahun Baru, Kapolda Metro Jaya Bonceng Anak Pakai Motor Trail",kompas.com,"[Pantau, Tahun, Baru,, Kapolda, Metro, Jaya, Bonceng, Anak, Pakai, Motor, Trail]","Pantau Tahun Baru, Kapolda Metro Jaya Bonceng Anak Pakai Motor Trail","[Pantau, Tahun, Baru, ,, Kapolda, Metro, Jaya, Bonceng, Anak, Pakai, Motor, Trail]","[pantau, tahun, baru, ,, kapolda, metro, jaya, bonceng, anak, pakai, motor, trail]","[pantau, tahun, baru, kapolda, metro, jaya, bonceng, anak, pakai, motor, trail]","[pantau, kapolda, metro, jaya, bonceng, anak, pakai, motor, trail]","[(pantau, NN), (kapolda, FW), (metro, NN), (jaya, NN), (bonceng, NN), (anak, NN), (pakai, NN), (...","[(pantau, n), (kapolda, n), (metro, n), (jaya, n), (bonceng, n), (anak, n), (pakai, n), (motor, ..."
3,5af9bde7421aa98c745d28cd,Terompet dan Kembang Api Bersahutan Sambut Tahun Baru di Monas,liputan6.com,"[Terompet, dan, Kembang, Api, Bersahutan, Sambut, Tahun, Baru, di, Monas]",Terompet dan Kembang Api Bersahutan Sambut Tahun Baru di Monas,"[Terompet, dan, Kembang, Api, Bersahutan, Sambut, Tahun, Baru, di, Monas]","[terompet, dan, kembang, api, bersahutan, sambut, tahun, baru, di, monas]","[terompet, dan, kembang, api, bersahutan, sambut, tahun, baru, di, monas]","[terompet, kembang, api, bersahutan, sambut, monas]","[(terompet, NN), (kembang, VBD), (api, JJ), (bersahutan, NN), (sambut, NN), (monas, NNS)]","[(terompet, n), (kembang, v), (api, a), (bersahutan, n), (sambut, n), (monas, n)]"
4,5af9bde4421aa98c745d28cc,Awas! Copet Berkeliaran di Ancol Malam Tahun Baru Ini,liputan6.com,"[Awas!, Copet, Berkeliaran, di, Ancol, Malam, Tahun, Baru, Ini]",Awas! Copet Berkeliaran di Ancol Malam Tahun Baru Ini,"[Awas, !, Copet, Berkeliaran, di, Ancol, Malam, Tahun, Baru, Ini]","[awas, !, copet, berkeliaran, di, ancol, malam, tahun, baru, ini]","[awas, copet, berkeliaran, di, ancol, malam, tahun, baru, ini]","[awas, copet, berkeliaran, ancol, malam]","[(awas, NN), (copet, NN), (berkeliaran, NN), (ancol, NN), (malam, NN)]","[(awas, n), (copet, n), (berkeliaran, n), (ancol, n), (malam, n)]"


In [86]:
#mengembalikan kata ke bentuk dasar
wnl = WordNetLemmatizer()
data3['lemmatized'] = data3['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])

In [87]:
data3.head()

,item_id,title,source,no_contract,title_str,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos,lemmatized
0,5af9bdef421aa98c745d28cf,Tips Agar Resolusi Tahun Baru Bisa Menjadi Aksi Nyata,liputan6.com,"[Tips, Agar, Resolusi, Tahun, Baru, Bisa, Menjadi, Aksi, Nyata]",Tips Agar Resolusi Tahun Baru Bisa Menjadi Aksi Nyata,"[Tips, Agar, Resolusi, Tahun, Baru, Bisa, Menjadi, Aksi, Nyata]","[tips, agar, resolusi, tahun, baru, bisa, menjadi, aksi, nyata]","[tips, agar, resolusi, tahun, baru, bisa, menjadi, aksi, nyata]","[tips, resolusi, aksi, nyata]","[(tips, NNS), (resolusi, VBP), (aksi, NN), (nyata, NNS)]","[(tips, n), (resolusi, v), (aksi, n), (nyata, n)]","[tip, resolusi, aksi, nyata]"
1,5af9bdeb421aa98c745d28ce,Tragedi Mendebarkan di Penghujung,liputan6.com,"[Tragedi, Mendebarkan, di, Penghujung]",Tragedi Mendebarkan di Penghujung,"[Tragedi, Mendebarkan, di, Penghujung]","[tragedi, mendebarkan, di, penghujung]","[tragedi, mendebarkan, di, penghujung]","[tragedi, mendebarkan, penghujung]","[(tragedi, NN), (mendebarkan, NN), (penghujung, NN)]","[(tragedi, n), (mendebarkan, n), (penghujung, n)]","[tragedi, mendebarkan, penghujung]"
2,5af85c6b421aa922e201b8e2,"Pantau Tahun Baru, Kapolda Metro Jaya Bonceng Anak Pakai Motor Trail",kompas.com,"[Pantau, Tahun, Baru,, Kapolda, Metro, Jaya, Bonceng, Anak, Pakai, Motor, Trail]","Pantau Tahun Baru, Kapolda Metro Jaya Bonceng Anak Pakai Motor Trail","[Pantau, Tahun, Baru, ,, Kapolda, Metro, Jaya, Bonceng, Anak, Pakai, Motor, Trail]","[pantau, tahun, baru, ,, kapolda, metro, jaya, bonceng, anak, pakai, motor, trail]","[pantau, tahun, baru, kapolda, metro, jaya, bonceng, anak, pakai, motor, trail]","[pantau, kapolda, metro, jaya, bonceng, anak, pakai, motor, trail]","[(pantau, NN), (kapolda, FW), (metro, NN), (jaya, NN), (bonceng, NN), (anak, NN), (pakai, NN), (...","[(pantau, n), (kapolda, n), (metro, n), (jaya, n), (bonceng, n), (anak, n), (pakai, n), (motor, ...","[pantau, kapolda, metro, jaya, bonceng, anak, pakai, motor, trail]"
3,5af9bde7421aa98c745d28cd,Terompet dan Kembang Api Bersahutan Sambut Tahun Baru di Monas,liputan6.com,"[Terompet, dan, Kembang, Api, Bersahutan, Sambut, Tahun, Baru, di, Monas]",Terompet dan Kembang Api Bersahutan Sambut Tahun Baru di Monas,"[Terompet, dan, Kembang, Api, Bersahutan, Sambut, Tahun, Baru, di, Monas]","[terompet, dan, kembang, api, bersahutan, sambut, tahun, baru, di, monas]","[terompet, dan, kembang, api, bersahutan, sambut, tahun, baru, di, monas]","[terompet, kembang, api, bersahutan, sambut, monas]","[(terompet, NN), (kembang, VBD), (api, JJ), (bersahutan, NN), (sambut, NN), (monas, NNS)]","[(terompet, n), (kembang, v), (api, a), (bersahutan, n), (sambut, n), (monas, n)]","[terompet, kembang, api, bersahutan, sambut, mona]"
4,5af9bde4421aa98c745d28cc,Awas! Copet Berkeliaran di Ancol Malam Tahun Baru Ini,liputan6.com,"[Awas!, Copet, Berkeliaran, di, Ancol, Malam, Tahun, Baru, Ini]",Awas! Copet Berkeliaran di Ancol Malam Tahun Baru Ini,"[Awas, !, Copet, Berkeliaran, di, Ancol, Malam, Tahun, Baru, Ini]","[awas, !, copet, berkeliaran, di, ancol, malam, tahun, baru, ini]","[awas, copet, berkeliaran, di, ancol, malam, tahun, baru, ini]","[awas, copet, berkeliaran, ancol, malam]","[(awas, NN), (copet, NN), (berkeliaran, NN), (ancol, NN), (malam, NN)]","[(awas, n), (copet, n), (berkeliaran, n), (ancol, n), (malam, n)]","[awas, copet, berkeliaran, ancol, malam]"
